In [1]:
from shapely.geometry import Point
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import folium
from folium.plugins import MarkerCluster
from folium.plugins import FastMarkerCluster

#### Let's load the Nashville zipcode data and look at the first few rows
- geopandas reads files with the `read_file()` method
- The zipcode data was downloaded from https://data.nashville.gov/General-Government/Zip-Codes/u7r5-bpku

In [2]:
zipcodes = gpd.read_file('../data/zipcodes.geojson')
print(zipcodes.crs)
zipcodes.head( )

epsg:4326


,zip,objectid,po_name,shape_stlength,shape_starea,geometry
0,37115,1,MADISON,178783.02488886821,596553400.57885742,"MULTIPOLYGON (((-86.68725 36.31821, -86.68722 ..."
1,37216,3,NASHVILLE,75820.997821400058,188884682.28344727,"MULTIPOLYGON (((-86.73451 36.23774, -86.73425 ..."
2,37204,9,NASHVILLE,93180.292250425613,200664795.51708984,"MULTIPOLYGON (((-86.77914 36.13424, -86.77923 ..."
3,37027,11,BRENTWOOD,159760.6942933173,174978422.04101562,"MULTIPOLYGON (((-86.81258 36.06319, -86.81263 ..."
4,37064,18,FRANKLIN,28995.828320601937,46969608.005737305,"MULTIPOLYGON (((-87.02197 36.01200, -87.02140 ..."


In [3]:
type(zipcodes)

geopandas.geodataframe.GeoDataFrame

#### Note that _if_ the crs contains init it needs to be [updated](https://geopandas.org/projections.html) as shown in the cell below

In [4]:
# zipcodes.geometry.to_crs('EPSG:4326')
# print(zipcodes.crs)

#### Next you will read in the historical markers data
- downloaded from https://data.nashville.gov/History/Historical-Markers/vk65-u7my
- print the shape 
- take a look at the first 3 rows

In [5]:
historical_markers = pd.read_csv('../data/Historical_Markers.csv')
print(historical_markers.shape)
historical_markers.head(3)

(220, 11)


,Number,Year Erected,Marker Name,Marker Text,Civil War Site?,Notes,Location,Latitude,Longitude,Condition,Mapped Location
0,6,1968.0,Battle of Nashville Stewart's Line,"Loring's division of Stewart's Corps, Hood's C...",True,NaN,4618 Lealand Lane,36.086311,-86.791167,NaN,POINT (-86.791167 36.086311)
1,90,1992.0,University School of Nashville,Founded in 1915 as the successor to The Winthr...,NaN,NaN,2000 Edgehill Avenue,36.143990,-86.798228,NaN,POINT (-86.798228 36.14399)
2,168,2018.0,May-granbery House and Alford Cemetery,Revolutionary War Pvt. John Alford built a two...,NaN,Council District 4,621 Hill Road,36.047802,-86.754991,NaN,POINT (-86.754991 36.047802)


In [6]:
historical_markers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 220 entries, 0 to 219
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Number           220 non-null    int64  
 1   Year Erected     215 non-null    float64
 2   Marker Name      220 non-null    object 
 3   Marker Text      220 non-null    object 
 4   Civil War Site?  11 non-null     object 
 5   Notes            110 non-null    object 
 6   Location         218 non-null    object 
 7   Latitude         216 non-null    float64
 8   Longitude        216 non-null    float64
 9   Condition        18 non-null     object 
 10  Mapped Location  216 non-null    object 
dtypes: float64(3), int64(1), object(7)
memory usage: 19.0+ KB


#### In order to spatially join the bus stops with the zipcodes, you will need to create a GeoDataFrame of bus stops
- First create a geometry column that is a `Point` datatype

In [7]:
#putting x.lng, x.lat together makes them a tuple

historical_markers['geometry'] = historical_markers.apply(lambda x: Point((x.Longitude, 
                                                         x.Latitude)), 
                                        axis=1)
historical_markers.head(3)

,Number,Year Erected,Marker Name,Marker Text,Civil War Site?,Notes,Location,Latitude,Longitude,Condition,Mapped Location,geometry
0,6,1968.0,Battle of Nashville Stewart's Line,"Loring's division of Stewart's Corps, Hood's C...",True,NaN,4618 Lealand Lane,36.086311,-86.791167,NaN,POINT (-86.791167 36.086311),POINT (-86.791167 36.086311)
1,90,1992.0,University School of Nashville,Founded in 1915 as the successor to The Winthr...,NaN,NaN,2000 Edgehill Avenue,36.143990,-86.798228,NaN,POINT (-86.798228 36.14399),POINT (-86.79822799999999 36.14399)
2,168,2018.0,May-granbery House and Alford Cemetery,Revolutionary War Pvt. John Alford built a two...,NaN,Council District 4,621 Hill Road,36.047802,-86.754991,NaN,POINT (-86.754991 36.047802),POINT (-86.754991 36.047802)


In [8]:
historical_markers = historical_markers.drop(['Mapped Location'], axis=1)
historical_markers.head()

,Number,Year Erected,Marker Name,Marker Text,Civil War Site?,Notes,Location,Latitude,Longitude,Condition,geometry
0,6,1968.0,Battle of Nashville Stewart's Line,"Loring's division of Stewart's Corps, Hood's C...",True,NaN,4618 Lealand Lane,36.086311,-86.791167,NaN,POINT (-86.791167 36.086311)
1,90,1992.0,University School of Nashville,Founded in 1915 as the successor to The Winthr...,NaN,NaN,2000 Edgehill Avenue,36.143990,-86.798228,NaN,POINT (-86.79822799999999 36.14399)
2,168,2018.0,May-granbery House and Alford Cemetery,Revolutionary War Pvt. John Alford built a two...,NaN,Council District 4,621 Hill Road,36.047802,-86.754991,NaN,POINT (-86.754991 36.047802)
3,19,1969.0,Meharry Medical College,"Meharry Medical College, established in 1876 t...",NaN,replaced 2012,"Meharry Medical College, D.B. Todd Boulvard en...",36.166770,-86.805937,NaN,POINT (-86.805937 36.16677)
4,29,1970.0,Madison College,Madison College was founded in 1904 as Nashvil...,NaN,NaN,"Hospital Drive off of Neely's Bend Road, Madison",36.250539,-86.683834,NaN,POINT (-86.683834 36.250539)


#### We can use the geometry field to build a GeoDataFrame 
- you want the CRS for the historical markers to match the CRS in the zipcodes GeoDataFrame 

In [9]:
zipcodes.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

#### The `GeoDataFrame()` constructor needs 3 parameters:
 - a DataFrame
 - a coordinate reference system
 - a geometry

In [11]:
hm_geo = gpd.GeoDataFrame(historical_markers, 
                           crs = zipcodes.crs, 
                           geometry = historical_markers['geometry'])

In [12]:
type(hm_geo)

geopandas.geodataframe.GeoDataFrame

#### Now you are ready to combine historical markers and zipcodes with a spatial join
- first a little clean up for the zipcodes
    - just keep the zip, po_name, and geometry columns
- then construct the join 
    - keep in mind that you want to find the bus stops within a zip code polygon
    - the geometry for the new (combined) GeoDataFrame comes from the left GeoDataFrame

In [13]:
zipcodes = zipcodes[['zip', 'po_name', 'geometry']]

In [14]:
#find historical markers within zipcodes
#sjoin = spatial join
hms_by_zip = gpd.sjoin(hm_geo, zipcodes, op = 'within')

In [15]:
hms_by_zip.head()

,Number,Year Erected,Marker Name,Marker Text,Civil War Site?,Notes,Location,Latitude,Longitude,Condition,geometry,index_right,zip,po_name
0,6,1968.0,Battle of Nashville Stewart's Line,"Loring's division of Stewart's Corps, Hood's C...",True,NaN,4618 Lealand Lane,36.086311,-86.791167,NaN,POINT (-86.79117 36.08631),53,37220,NASHVILLE
14,132,2008.0,Dry-stack Stone Walls,"Dry-stack stone walls, a Scots-Irish building ...",True,NaN,4708 Granny White Pike,36.085384,-86.804379,NaN,POINT (-86.80438 36.08538),53,37220,NASHVILLE
117,134,2008.0,"Battle of Nashville (december 16, 1864) Confed...",After the withdrawal from the main Confederate...,True,NaN,827 Tyne Blvd,36.077687,-86.777135,NaN,POINT (-86.77714 36.07769),53,37220,NASHVILLE
148,205,2019.0,Crieve Hall,The Crieve Hall neighborhood was part of the o...,NaN,Council District 26,498 Hogan Road,36.065264,-86.757741,NaN,POINT (-86.75774 36.06526),53,37220,NASHVILLE
181,28,1970.0,Granny White Grave,Lucinda 'Granny' White of N.C. acquired 50 acr...,NaN,Marker Project- prioirty list replaced with up...,Travelers Ridge Drive off of Granny White Pike,36.073301,-86.807104,NaN,POINT (-86.80710 36.07330),53,37220,NASHVILLE


#### Now that the data is joined spatially you can count the historical markers by zip code and see how many are within each polygon 
- using the `value_counts()` method